In [ ]:
from google.colab import drive
from google.colab import files
import shutil
import os

# Mount Google Drive to access it from Colab
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `unzip -q /content/val.zip -d /content/drive/MyDrive/dl_project_data/val (1)'
/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `unzip -q /content/test.zip -d /content/drive/MyDrive/dl_project_data/test (1)'
/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `unzip -q /content/train.zip -d /content/drive/MyDrive/dl_project_data/train (1)'


In [ ]:
!unzip    /content/val.zip -d /content/val
!unzip    /content/test.zip -d /content/test
!unzip  /content/train.zip -d /content/train


Archive:  /content/val.zip
   creating: /content/val/ApplyLipstick/
  inflating: /content/val/ApplyLipstick/v_ApplyLipstick_g01_c05.avi  
  inflating: /content/val/ApplyLipstick/v_ApplyLipstick_g07_c02.avi  
  inflating: /content/val/ApplyLipstick/v_ApplyLipstick_g09_c04.avi  
  inflating: /content/val/ApplyLipstick/v_ApplyLipstick_g10_c03.avi  
  inflating: /content/val/ApplyLipstick/v_ApplyLipstick_g11_c03.avi  
  inflating: /content/val/ApplyLipstick/v_ApplyLipstick_g13_c04.avi  
  inflating: /content/val/ApplyLipstick/v_ApplyLipstick_g14_c01.avi  
  inflating: /content/val/ApplyLipstick/v_ApplyLipstick_g15_c01.avi  
  inflating: /content/val/ApplyLipstick/v_ApplyLipstick_g15_c02.avi  
  inflating: /content/val/ApplyLipstick/v_ApplyLipstick_g16_c05.avi  
  inflating: /content/val/ApplyLipstick/v_ApplyLipstick_g18_c03.avi  
  inflating: /content/val/ApplyLipstick/v_ApplyLipstick_g19_c01.avi  
  inflating: /content/val/ApplyLipstick/v_ApplyLipstick_g23_c01.avi  
  inflating: /content/

In [ ]:
!nvidia-smi


Wed Dec  4 04:49:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


CUDA available: True


In [ ]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


PyTorch version: 2.5.1+cu121
CUDA available: True


In [ ]:
import pandas as pd
import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
from PIL import Image
from skimage.metrics import structural_similarity as ssim
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt

# Step 1: Load Data
train_df = pd.read_csv("/content/Train.csv")
test_df = pd.read_csv("/content/Test.csv")
eval_df = pd.read_csv("/content/Val.csv")

# Update paths for Colab
eval_df["clip_path"] = "/content/" + eval_df["clip_path"]
test_df["clip_path"] = "/content/" + test_df["clip_path"]
train_df["clip_path"] = "/content/" + train_df["clip_path"]

# Step 2: Preprocess Labels
label_encoder = LabelEncoder()
train_df["label_encoded"] = label_encoder.fit_transform(train_df["label"])
test_df["label_encoded"] = label_encoder.transform(test_df["label"])
eval_df["label_encoded"] = label_encoder.transform(eval_df["label"])

train_paths = train_df["clip_path"].values
train_labels = train_df["label_encoded"].values

test_paths = test_df["clip_path"].values
test_labels = test_df["label_encoded"].values

eval_paths = eval_df["clip_path"].values
eval_labels = eval_df["label_encoded"].values

# Step 3: Dataset and DataLoader
class VideoDataset(Dataset):
    def __init__(self, paths, labels, transform=None, input_frames=10, pred_frames=5):
        self.paths = paths
        self.labels = labels
        self.transform = transform
        self.input_frames = input_frames
        self.pred_frames = pred_frames

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        video_path = self.paths[idx]
        label = self.labels[idx]

        # Extract frames
        frames = self._extract_frames(video_path, self.input_frames + self.pred_frames)
        if len(frames) < self.input_frames + self.pred_frames:
            frames += [frames[-1]] * (self.input_frames + self.pred_frames - len(frames))

        # Apply transformations
        if self.transform:
            frames = [self.transform(frame) for frame in frames]
        frames = torch.stack(frames)  # [T, C, H, W]

        input_frames = frames[:self.input_frames]  # Input sequence
        pred_frames = frames[self.input_frames:]  # Ground truth prediction
        return input_frames, pred_frames

    def _extract_frames(self, video_path, total_frames):
        cap = cv2.VideoCapture(video_path)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        step = max(1, frame_count // total_frames)
        frames = []
        for i in range(0, frame_count, step):
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = Image.fromarray(frame)
            frames.append(frame)
        cap.release()
        return frames[:total_frames]

# Transformations (64x64 as required)
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Grayscale
])

# Create datasets and dataloaders
train_dataset = VideoDataset(train_paths, train_labels, transform=transform)
test_dataset = VideoDataset(test_paths, test_labels, transform=transform)
eval_dataset = VideoDataset(eval_paths, eval_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)
eval_loader = DataLoader(eval_dataset, batch_size=4, shuffle=False)

# Step 4: Define the ConvLSTM Model
class ConvLSTM(nn.Module):
    def __init__(self, input_frames, pred_frames, num_classes=1):
        super(ConvLSTM, self).__init__()
        self.conv1 = nn.Conv3d(3, 32, kernel_size=(3, 3, 3), stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool3d((1, 2, 2))  # Reduces spatial dimensions (H, W) by half
        self.lstm = nn.LSTM(input_size=32 * 32 * 32, hidden_size=256, batch_first=True)
        self.fc = nn.Linear(256, pred_frames * 32 * 32 * 3)
        self.upsample = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=False)  # Restore to 64x64

    def forward(self, x):
        # Rearrange to match Conv3D input expectations
        x = x.permute(0, 2, 1, 3, 4)  # Shape: [batch_size, channels, depth, height, width]

        # Convolutional layers
        x = self.pool(self.relu(self.conv1(x)))  # Conv3D + Pooling

        # Flatten for LSTM
        b, c, t, h, w = x.size()
        x = x.view(b, t, -1)  # Shape: [batch_size, depth, features]

        # LSTM
        x, _ = self.lstm(x)  # Shape: [batch_size, depth, hidden_size]
        x = self.fc(x[:, -1, :])  # Use the last LSTM output

        # Reshape and upsample
        x = x.view(b, -1, 3, 32, 32)  # Shape: [batch_size, pred_frames, channels, 32, 32]
        x = x.view(-1, 3, 32, 32)  # Flatten pred_frames for upsampling
        x = self.upsample(x)  # Upsample to [batch_size * pred_frames, 3, 64, 64]
        x = x.view(b, -1, 3, 64, 64)  # Restore batch and pred_frames structure
        return x

# Initialize model, loss, optimizer
input_frames = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pred_frames = 5
model = ConvLSTM(input_frames, pred_frames).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 5: Train the Model
def train_model(model, train_loader, criterion, optimizer, epochs=200, save_path="best_model.pth"):
    best_loss = float('inf')
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for input_frames, target_frames in train_loader:
            input_frames, target_frames = input_frames.to(device), target_frames.to(device)
            optimizer.zero_grad()
            pred_frames = model(input_frames)
            loss = criterion(pred_frames, target_frames)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
        if avg_loss < best_loss:
            best_loss = avg_loss
            torch.save(model.state_dict(), save_path)
            print(f"Model saved to {save_path}")

# Step 6: Evaluate Model with MSE and SSIM




# Train and Evaluate
train_model(model, train_loader, criterion, optimizer, epochs=50)



Epoch 1/50, Loss: 0.1999
Model saved to best_model.pth
Epoch 2/50, Loss: 0.1438
Model saved to best_model.pth
Epoch 3/50, Loss: 0.1237
Model saved to best_model.pth
Epoch 4/50, Loss: 0.1119
Model saved to best_model.pth
Epoch 5/50, Loss: 0.1058
Model saved to best_model.pth
Epoch 6/50, Loss: 0.0983
Model saved to best_model.pth
Epoch 7/50, Loss: 0.0945
Model saved to best_model.pth
Epoch 8/50, Loss: 0.0902
Model saved to best_model.pth
Epoch 9/50, Loss: 0.0859
Model saved to best_model.pth
Epoch 10/50, Loss: 0.0830
Model saved to best_model.pth
Epoch 11/50, Loss: 0.0807
Model saved to best_model.pth
Epoch 12/50, Loss: 0.0770
Model saved to best_model.pth
Epoch 13/50, Loss: 0.0737
Model saved to best_model.pth
Epoch 14/50, Loss: 0.0709
Model saved to best_model.pth
Epoch 15/50, Loss: 0.0687
Model saved to best_model.pth
Epoch 16/50, Loss: 0.0672
Model saved to best_model.pth
Epoch 17/50, Loss: 0.0643
Model saved to best_model.pth
Epoch 18/50, Loss: 0.0627
Model saved to best_model.pth
E

In [ ]:
import pandas as pd
import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
from PIL import Image
from skimage.metrics import structural_similarity as ssim
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt

# Step 1: Load Data
train_df = pd.read_csv("/content/Train.csv")
test_df = pd.read_csv("/content/Test.csv")
eval_df = pd.read_csv("/content/Val.csv")

# Update paths for Colab
eval_df["clip_path"] = "/content/" + eval_df["clip_path"]
test_df["clip_path"] = "/content/" + test_df["clip_path"]
train_df["clip_path"] = "/content/" + train_df["clip_path"]

# Step 2: Dataset and DataLoader
class VideoDataset(Dataset):
    def __init__(self, paths, labels, transform=None, input_frames=10, pred_frames=5):
        self.paths = paths
        self.labels = labels
        self.transform = transform
        self.input_frames = input_frames
        self.pred_frames = pred_frames

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        video_path = self.paths[idx]
        label = self.labels[idx]
        frames = self._extract_frames(video_path, self.input_frames + self.pred_frames)
        if len(frames) < self.input_frames + self.pred_frames:
            frames += [frames[-1]] * (self.input_frames + self.pred_frames - len(frames))
        if self.transform:
            frames = [self.transform(frame) for frame in frames]
        frames = torch.stack(frames)  # [T, C, H, W]
        input_frames = frames[:self.input_frames]
        pred_frames = frames[self.input_frames:]
        return input_frames, pred_frames

    def _extract_frames(self, video_path, total_frames):
        cap = cv2.VideoCapture(video_path)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        step = max(1, frame_count // total_frames)
        frames = []
        for i in range(0, frame_count, step):
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = Image.fromarray(frame)
            frames.append(frame)
        cap.release()
        return frames[:total_frames]

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])
# Step 2: Preprocess Labels
label_encoder = LabelEncoder()
train_df["label_encoded"] = label_encoder.fit_transform(train_df["label"])
test_df["label_encoded"] = label_encoder.transform(test_df["label"])
eval_df["label_encoded"] = label_encoder.transform(eval_df["label"])

train_paths = train_df["clip_path"].values
train_labels = train_df["label_encoded"].values

test_paths = test_df["clip_path"].values
test_labels = test_df["label_encoded"].values

eval_paths = eval_df["clip_path"].values
eval_labels = eval_df["label_encoded"].values


# Create datasets and dataloaders
train_dataset = VideoDataset(train_paths, train_labels, transform=transform)
test_dataset = VideoDataset(test_paths, test_labels, transform=transform)
eval_dataset = VideoDataset(eval_paths, eval_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)
eval_loader = DataLoader(eval_dataset, batch_size=4, shuffle=False)


In [ ]:
print("Class mappings:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))


Class mappings: {'ApplyEyeMakeup': 0, 'ApplyLipstick': 1, 'Archery': 2, 'BabyCrawling': 3, 'BalanceBeam': 4}


In [ ]:

target_class = "Archery"
class_idx = label_encoder.transform([target_class])[0]
filtered_df = eval_df[eval_df["label_encoded"] == class_idx]

# Extract paths and labels for this class
filtered_paths = filtered_df["clip_path"].values
filtered_labels = filtered_df["label_encoded"].values


In [ ]:
filtered_dataset = VideoDataset(filtered_paths, filtered_labels, transform=transform)
filtered_loader = DataLoader(filtered_dataset, batch_size=4, shuffle=False)


In [ ]:
def generate_video(frames, save_path="generated_video.mp4", fps=10):
    # Check input frames
    if len(frames) == 0:
        print("Error: No frames to write!")
        return
    if frames.max() <= 1.0:
        frames = np.clip((frames * 255), 0, 255).astype(np.uint8)  # Scale to [0, 255]

    # Ensure frames are in [T, H, W, C] format
    print(f"Frames to write: {len(frames)}, Resolution: {frames[0].shape[:2]}")

    height, width = frames[0].shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(save_path, fourcc, fps, (width, height))

    # Write frames to video
    for i, frame in enumerate(frames):
        print(f"Writing frame {i+1}/{len(frames)}")
        video_writer.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))  # OpenCV expects BGR

    video_writer.release()
    print(f"Video saved to {save_path}")
# Load trained model
# Load trained model
model.load_state_dict(torch.load("/content/best_model.pth"))# Evaluate on filtered class
model.eval()
for batch_idx, (input_frames, target_frames) in enumerate(filtered_loader):
    input_frames = input_frames.to(device)
    with torch.no_grad():
        pred_frames_batch = model(input_frames).cpu().numpy()  # [batch_size, T, C, H, W]

    # Save videos
    for i, pred_frames in enumerate(pred_frames_batch):
        # Process the predicted frames
        pred_frames = pred_frames.transpose(0, 2, 3, 1)  # [T, H, W, C]
        pred_frames = (pred_frames * 0.5) + 0.5  # Convert to [0, 1]
        pred_frames = np.clip(pred_frames, 0, 1)

        # Save the video
        save_path = f"predicted_{target_class}_video_{batch_idx*4 + i + 1}.mp4"
        generate_video(pred_frames, save_path=save_path, fps=10)
        print(f"Saved: {save_path}")





<ipython-input-13-308fd41d789c>:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/content/best_model.pth"))# Evaluate on filtered class


Frames to write: 5, Resolution: (64, 64)
Writing frame 1/5
Writing frame 2/5
Writing frame 3/5
Writing frame 4/5
Writing frame 5/5
Video saved to predicted_Archery_video_1.mp4
Saved: predicted_Archery_video_1.mp4
Frames to write: 5, Resolution: (64, 64)
Writing frame 1/5
Writing frame 2/5
Writing frame 3/5
Writing frame 4/5
Writing frame 5/5
Video saved to predicted_Archery_video_2.mp4
Saved: predicted_Archery_video_2.mp4
Frames to write: 5, Resolution: (64, 64)
Writing frame 1/5
Writing frame 2/5
Writing frame 3/5
Writing frame 4/5
Writing frame 5/5
Video saved to predicted_Archery_video_3.mp4
Saved: predicted_Archery_video_3.mp4
Frames to write: 5, Resolution: (64, 64)
Writing frame 1/5
Writing frame 2/5
Writing frame 3/5
Writing frame 4/5
Writing frame 5/5
Video saved to predicted_Archery_video_4.mp4
Saved: predicted_Archery_video_4.mp4
Frames to write: 5, Resolution: (64, 64)
Writing frame 1/5
Writing frame 2/5
Writing frame 3/5
Writing frame 4/5
Writing frame 5/5
Video saved to pr

In [ ]:
from skimage.metrics import structural_similarity as ssim
import numpy as np
import torch

def evaluate_model(model, data_loader, device, load_path=None):
    if load_path:  # Load model if a path is provided
        model.load_state_dict(torch.load(load_path, map_location=device))
        print(f"Model loaded from {load_path}")

    model.eval()
    total_mse, total_ssim = 0, 0

    with torch.no_grad():
        for input_frames, target_frames in data_loader:
            input_frames, target_frames = input_frames.to(device), target_frames.to(device)
            pred_frames = model(input_frames).cpu().numpy()  # [batch_size, pred_frames, channels, height, width]
            target_frames = target_frames.cpu().numpy()

            # Iterate over batch
            for pred_seq, target_seq in zip(pred_frames, target_frames):  # [pred_frames, channels, height, width]
                for pred, target in zip(pred_seq, target_seq):  # Iterate over individual frames
                    # Normalize and convert to [0, 1]
                    pred = np.clip((pred * 0.5) + 0.5, 0, 1)
                    target = np.clip((target * 0.5) + 0.5, 0, 1)

                    # Transpose to (height, width, channels) for SSIM
                    pred = pred.transpose(1, 2, 0)
                    target = target.transpose(1, 2, 0)

                    # Calculate MSE
                    mse = ((pred - target) ** 2).mean()
                    total_mse += mse

                    # Adjust win_size dynamically for SSIM
                    win_size = min(7, pred.shape[0], pred.shape[1])  # Ensure win_size fits dimensions
                    if win_size % 2 == 0:  # Ensure odd win_size
                        win_size -= 1

                    # Calculate SSIM
                    ssim_value = ssim(
                        pred, target,
                        data_range=1.0,  # Adjust based on normalization
                        win_size=win_size,
                        channel_axis=-1  # Specify channel axis
                    )
                    total_ssim += ssim_value

    # Compute averages
    num_frames = len(data_loader.dataset) * pred_frames.shape[1]  # Total number of frames
    avg_mse = total_mse / num_frames
    avg_ssim = total_ssim / num_frames

    print(f"Evaluation Results: MSE: {avg_mse:.4f}, SSIM: {avg_ssim:.4f}")

# After training, evaluate the model on the test or validation dataset
evaluate_model(model, eval_loader, device, load_path="best_model.pth")


<ipython-input-14-f6d14f34a671>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(load_path, map_location=device))


Model loaded from best_model.pth
Evaluation Results: MSE: 0.0158, SSIM: 0.4520


In [ ]:
from skimage.metrics import structural_similarity as ssim
import numpy as np
import torch

def evaluate_model(model, data_loader, device, dataset_name="Validation", load_path=None):
    """
    Evaluate the model on a given dataset loader.

    Args:
    - model: PyTorch model to be evaluated.
    - data_loader: DataLoader for the dataset.
    - device: PyTorch device (e.g., "cuda" or "cpu").
    - dataset_name: String indicating the dataset type (e.g., "Train", "Test").
    - load_path: Path to load a pretrained model. Default is None.

    Returns:
    - avg_mse: Average Mean Squared Error over the dataset.
    - avg_ssim: Average Structural Similarity Index over the dataset.
    """
    if load_path:  # Load model if a path is provided
        model.load_state_dict(torch.load(load_path, map_location=device))
        print(f"Model loaded from {load_path}")

    model.eval()
    total_mse, total_ssim = 0, 0

    with torch.no_grad():
        for input_frames, target_frames in data_loader:
            input_frames, target_frames = input_frames.to(device), target_frames.to(device)
            pred_frames = model(input_frames).cpu().numpy()  # [batch_size, pred_frames, channels, height, width]
            target_frames = target_frames.cpu().numpy()

            # Iterate over batch
            for pred_seq, target_seq in zip(pred_frames, target_frames):  # [pred_frames, channels, height, width]
                for pred, target in zip(pred_seq, target_seq):  # Iterate over individual frames
                    # Normalize and convert to [0, 1]
                    pred = np.clip((pred * 0.5) + 0.5, 0, 1)
                    target = np.clip((target * 0.5) + 0.5, 0, 1)

                    # Transpose to (height, width, channels) for SSIM
                    pred = pred.transpose(1, 2, 0)
                    target = target.transpose(1, 2, 0)

                    # Calculate MSE
                    mse = ((pred - target) ** 2).mean()
                    total_mse += mse

                    # Adjust win_size dynamically for SSIM
                    win_size = min(7, pred.shape[0], pred.shape[1])  # Ensure win_size fits dimensions
                    if win_size % 2 == 0:  # Ensure odd win_size
                        win_size -= 1

                    # Calculate SSIM
                    ssim_value = ssim(
                        pred, target,
                        data_range=1.0,  # Adjust based on normalization
                        win_size=win_size,
                        channel_axis=-1  # Specify channel axis
                    )
                    total_ssim += ssim_value

    # Compute averages
    num_frames = len(data_loader.dataset) * pred_frames.shape[1]  # Total number of frames
    avg_mse = total_mse / num_frames
    avg_ssim = total_ssim / num_frames

    print(f"{dataset_name} Results: MSE: {avg_mse:.4f}, SSIM: {avg_ssim:.4f}")
    return avg_mse, avg_ssim

# Evaluate the model on train and test datasets
train_results = evaluate_model(model, train_loader, device, dataset_name="Train", load_path="best_model.pth")
test_results = evaluate_model(model, test_loader, device, dataset_name="Test")


<ipython-input-15-1e2411b55f86>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(load_path, map_location=device))


Model loaded from best_model.pth
Train Results: MSE: 0.0076, SSIM: 0.6183
Test Results: MSE: 0.0171, SSIM: 0.4278


In [ ]:
from skimage.metrics import structural_similarity as ssim
import numpy as np
import torch

def evaluate_model(model, data_loader, device, dataset_name="Validation", load_path=None):
    """
    Evaluate the model on a given dataset loader.

    Args:
    - model: PyTorch model to be evaluated.
    - data_loader: DataLoader for the dataset.
    - device: PyTorch device (e.g., "cuda" or "cpu").
    - dataset_name: String indicating the dataset type (e.g., "Train", "Test").
    - load_path: Path to load a pretrained model. Default is None.

    Returns:
    - avg_mse: Average Mean Squared Error over the dataset.
    - avg_ssim: Average Structural Similarity Index over the dataset.
    """
    if load_path:  # Load model if a path is provided
        model.load_state_dict(torch.load(load_path, map_location=device))
        print(f"Model loaded from {load_path}")

    model.eval()
    total_mse, total_ssim = 0, 0

    with torch.no_grad():
        for input_frames, target_frames in data_loader:
            input_frames, target_frames = input_frames.to(device), target_frames.to(device)
            pred_frames = model(input_frames).cpu().numpy()  # [batch_size, pred_frames, channels, height, width]
            target_frames = target_frames.cpu().numpy()

            # Iterate over batch
            for pred_seq, target_seq in zip(pred_frames, target_frames):  # [pred_frames, channels, height, width]
                for pred, target in zip(pred_seq, target_seq):  # Iterate over individual frames
                    # Normalize and convert to [0, 1]
                    pred = np.clip((pred * 0.5) + 0.5, 0, 1)
                    target = np.clip((target * 0.5) + 0.5, 0, 1)

                    # Transpose to (height, width, channels) for SSIM
                    pred = pred.transpose(1, 2, 0)
                    target = target.transpose(1, 2, 0)

                    # Calculate MSE
                    mse = ((pred - target) ** 2).mean()
                    total_mse += mse

                    # Adjust win_size dynamically for SSIM
                    win_size = min(7, pred.shape[0], pred.shape[1])  # Ensure win_size fits dimensions
                    if win_size % 2 == 0:  # Ensure odd win_size
                        win_size -= 1

                    # Calculate SSIM
                    ssim_value = ssim(
                        pred, target,
                        data_range=1.0,  # Adjust based on normalization
                        win_size=win_size,
                        channel_axis=-1  # Specify channel axis
                    )
                    total_ssim += ssim_value

    # Compute averages
    num_frames = len(data_loader.dataset) * pred_frames.shape[1]  # Total number of frames
    avg_mse = total_mse / num_frames
    avg_ssim = total_ssim / num_frames

    print(f"{dataset_name} Results: MSE: {avg_mse:.4f}, SSIM: {avg_ssim:.4f}")
    return avg_mse, avg_ssim

# Evaluate the model on train, validation, and test datasets
train_results = evaluate_model(model, train_loader, device, dataset_name="Train", load_path="best_transformer_model.pth")
val_results = evaluate_model(model, eval_loader, device, dataset_name="Validation", load_path="best_transformer_model.pth")
test_results = evaluate_model(model, test_loader, device, dataset_name="Test")


In [ ]:
def combine_input_and_predicted_frames(input_frames, pred_frames):
    """
    Combine input frames and predicted frames into a single sequence for video generation.
    Args:
        input_frames: The original input frames (tensor of shape [T, C, H, W]).
        pred_frames: The predicted frames (tensor of shape [T, C, H, W]).

    Returns:
        combined_frames: A numpy array of combined frames in [T, H, W, C] format.
    """
    # Transpose input frames to [T, H, W, C] format
    input_frames = input_frames.permute(0, 2, 3, 1).cpu().numpy()  # [T, H, W, C]

    # Transpose predicted frames to [T, H, W, C] format
    pred_frames = pred_frames.permute(0, 2, 3, 1).cpu().numpy()  # [T, H, W, C]

    # Combine the input and predicted frames
    combined_frames = np.concatenate((input_frames, pred_frames), axis=0)  # Concatenate along time axis

    # Unnormalize and clip the frames
    combined_frames = (combined_frames * 0.5) + 0.5  # Convert to [0, 1]
    combined_frames = np.clip(combined_frames, 0, 1)  # Ensure valid range

    return combined_frames
# Load trained model
model.load_state_dict(torch.load("best_model.pth"))
model.eval()

# Get a batch from the evaluation set
input_frames, target_frames = next(iter(eval_loader))
input_frames = input_frames.to(device)

# Predict frames
with torch.no_grad():
    pred_frames = model(input_frames)
    print(f"Predicted frames shape: {pred_frames.shape}")  # [4, 5, 3, 64, 64]

# Select the first video in the batch
input_frames_single = input_frames[0]  # Shape: [10, 3, 64, 64] for the first video
pred_frames_single = pred_frames[0]  # Shape: [5, 3, 64, 64]

# Combine input and predicted frames
combined_frames = combine_input_and_predicted_frames(input_frames_single, pred_frames_single)

# Debug frame properties
print(f"Combined frames shape: {combined_frames.shape}")
print(f"Pixel value range after clipping: {combined_frames.min()} to {combined_frames.max()}")

# Generate video from combined frames
generate_video(combined_frames, save_path="combined_video.mp4", fps=10)

# Display video in Colab
from IPython.display import Video
Video("combined_video.mp4", embed=True)


<ipython-input-23-22ec34e129d4>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))


Predicted frames shape: torch.Size([4, 5, 3, 64, 64])
Combined frames shape: (15, 64, 64, 3)
Pixel value range after clipping: 0.0 to 0.9960784316062927
Frames to write: 15, Resolution: (64, 64)
Writing frame 1/15
Writing frame 2/15
Writing frame 3/15
Writing frame 4/15
Writing frame 5/15
Writing frame 6/15
Writing frame 7/15
Writing frame 8/15
Writing frame 9/15
Writing frame 10/15
Writing frame 11/15
Writing frame 12/15
Writing frame 13/15
Writing frame 14/15
Writing frame 15/15
Video saved to combined_video.mp4


PRED RNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the SpatioTemporalLSTMCell
class SpatioTemporalLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(SpatioTemporalLSTMCell, self).__init__()
        self.hidden_dim = hidden_dim

        # Note: input_dim only applies to the first layer; other layers use hidden_dim.
        self.input_to_state = nn.Conv2d(hidden_dim + input_dim, hidden_dim * 4, kernel_size=3, padding=1)
        self.state_to_memory = nn.Conv2d(hidden_dim * 2, hidden_dim, kernel_size=3, padding=1)

    def forward(self, input_tensor, hidden_state, cell_state, memory):
        # Concatenate input and hidden state
        combined = torch.cat([input_tensor, hidden_state], dim=1)  # Shape: [batch, input_dim + hidden_dim, H, W]

        # Compute gates
        gates = self.input_to_state(combined)  # Shape: [batch, hidden_dim * 4, H, W]
        i, f, o, g = torch.split(gates, self.hidden_dim, dim=1)  # Split into input, forget, output, and candidate gates
        i = torch.sigmoid(i)
        f = torch.sigmoid(f)
        o = torch.sigmoid(o)
        g = torch.tanh(g)

        # Update cell state and hidden state
        cell_state = f * cell_state + i * g
        hidden_state = o * torch.tanh(cell_state)

        # Update memory
        memory_input = torch.cat([hidden_state, cell_state], dim=1)
        memory = torch.tanh(self.state_to_memory(memory_input)) + memory

        return hidden_state, cell_state, memory

class PredRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, input_frames, pred_frames, height, width):
        super(PredRNN, self).__init__()
        self.input_frames = input_frames
        self.pred_frames = pred_frames
        self.height = height
        self.width = width

        # Initialize layers
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            in_channels = input_dim if i == 0 else hidden_dim
            self.layers.append(SpatioTemporalLSTMCell(in_channels, hidden_dim))

        # Output layer
        self.output_layer = nn.Conv2d(hidden_dim, input_dim, kernel_size=1)  # Final output to reconstruct frames

    def forward(self, input_frames):
        batch_size, seq_len, channels, height, width = input_frames.size()
        assert height == self.height and width == self.width, "Frame dimensions must match the model's height and width."

        # Initialize hidden states, cell states, and memory for each layer
        hidden_states = [torch.zeros(batch_size, layer.hidden_dim, self.height, self.width).to(input_frames.device)
                         for layer in self.layers]
        cell_states = [torch.zeros(batch_size, layer.hidden_dim, self.height, self.width).to(input_frames.device)
                       for layer in self.layers]
        memory = torch.zeros(batch_size, self.layers[0].hidden_dim, self.height, self.width).to(input_frames.device)

        # Process input frames
        for t in range(self.input_frames):
            x = input_frames[:, t]  # Extract frame at timestep t
            for i, layer in enumerate(self.layers):
                hidden_states[i], cell_states[i], memory = layer(
                    x if i == 0 else hidden_states[i - 1],  # Use input only for the first layer
                    hidden_states[i],
                    cell_states[i],
                    memory
                )

        # Generate predicted frames
        outputs = []
        for t in range(self.pred_frames):
            for i, layer in enumerate(self.layers):
                hidden_states[i], cell_states[i], memory = layer(
                    x if i == 0 else hidden_states[i - 1],  # Use output of previous layer
                    hidden_states[i],
                    cell_states[i],
                    memory
                )
            outputs.append(self.output_layer(hidden_states[-1]))  # Generate the output frame from the last layer

        return torch.stack(outputs, dim=1)  # Shape: [batch_size, pred_frames, channels, height, width]

transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to 64x64 pixels
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize to mean 0.5 and std 0.5
])
for input_frames, target_frames in train_loader:
    print(f"Input frames: {input_frames.shape}, Target frames: {target_frames.shape}")
    break


# Initialize the PredRNN model
input_frames = 10
pred_frames = 5
input_dim = 3  # RGB channels
hidden_dim = 64  # Hidden state dimension
num_layers = 2  # Number of ST-LSTM layers
height, width = 64, 64  # Frame dimensions
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PredRNN(input_dim, hidden_dim, num_layers, input_frames, pred_frames, height, width).to(device)





# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function
def train_model(model, train_loader, criterion, optimizer, epochs=50, save_path="best_predrnn_model.pth"):
    best_loss = float('inf')
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for input_frames, target_frames in train_loader:
            input_frames, target_frames = input_frames.to(device), target_frames.to(device)
            optimizer.zero_grad()
            pred_frames = model(input_frames)
            loss = criterion(pred_frames, target_frames)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
        if avg_loss < best_loss:
            best_loss = avg_loss
            torch.save(model.state_dict(), save_path)
            print(f"Model saved to {save_path}")
for input_frames, target_frames in train_loader:
    print(f"Input frames: {input_frames.shape}, Target frames: {target_frames.shape}")
    break

# Train the PredRNN model
#train_model(model, train_loader, criterion, optimizer, epochs=50)


Input frames: torch.Size([4, 10, 3, 64, 64]), Target frames: torch.Size([4, 5, 3, 64, 64])
Input frames: torch.Size([4, 10, 3, 64, 64]), Target frames: torch.Size([4, 5, 3, 64, 64])


In [ ]:
import torch
import numpy as np
from skimage.metrics import structural_similarity as ssim

def evaluate_model(model, data_loader, device, dataset_name="Validation", load_path=None):
    """
    Evaluate the model on a given dataset loader.

    Args:
    - model: PyTorch model to be evaluated.
    - data_loader: DataLoader for the dataset.
    - device: PyTorch device (e.g., "cuda" or "cpu").
    - dataset_name: String indicating the dataset type (e.g., "Train", "Test").
    - load_path: Path to load a pretrained model. Default is None.

    Returns:
    - avg_mse: Average Mean Squared Error over the dataset.
    - avg_ssim: Average Structural Similarity Index over the dataset.
    """
    if load_path:  # Load model if a path is provided
        model.load_state_dict(torch.load(load_path, map_location=device))
        print(f"Model loaded from {load_path}")

    model.eval()
    total_mse, total_ssim = 0, 0

    with torch.no_grad():
        for input_frames, target_frames in data_loader:
            input_frames, target_frames = input_frames.to(device), target_frames.to(device)
            pred_frames = model(input_frames).cpu().numpy()  # [batch_size, pred_frames, channels, height, width]
            target_frames = target_frames.cpu().numpy()

            # Iterate over batch
            for pred_seq, target_seq in zip(pred_frames, target_frames):  # [pred_frames, channels, height, width]
                for pred, target in zip(pred_seq, target_seq):  # Iterate over individual frames
                    # Normalize and convert to [0, 1]
                    pred = np.clip((pred * 0.5) + 0.5, 0, 1)
                    target = np.clip((target * 0.5) + 0.5, 0, 1)

                    # Transpose to (height, width, channels) for SSIM
                    pred = pred.transpose(1, 2, 0)
                    target = target.transpose(1, 2, 0)

                    # Calculate MSE
                    mse = ((pred - target) ** 2).mean()
                    total_mse += mse

                    # Adjust win_size dynamically for SSIM
                    win_size = min(7, pred.shape[0], pred.shape[1])  # Ensure win_size fits dimensions
                    if win_size % 2 == 0:  # Ensure odd win_size
                        win_size -= 1

                    # Calculate SSIM
                    ssim_value = ssim(
                        pred, target,
                        data_range=1.0,  # Adjust based on normalization
                        win_size=win_size,
                        channel_axis=-1  # Specify channel axis
                    )
                    total_ssim += ssim_value

    # Compute averages
    num_frames = len(data_loader.dataset) * pred_frames.shape[1]  # Total number of frames
    avg_mse = total_mse / num_frames
    avg_ssim = total_ssim / num_frames

    print(f"{dataset_name} Results: MSE: {avg_mse:.4f}, SSIM: {avg_ssim:.4f}")
    return avg_mse, avg_ssim

# Evaluate the model on train, validation, and test datasets
train_results = evaluate_model(model, train_loader, device, dataset_name="Train", load_path="best_predrnn_model.pth")
val_results = evaluate_model(model, eval_loader, device, dataset_name="Validation", load_path="best_predrnn_model.pth")
test_results = evaluate_model(model, test_loader, device, dataset_name="Test")

<ipython-input-19-8cb43ce010a6>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(load_path, map_location=device))


Model loaded from best_predrnn_model.pth
Train Results: MSE: 0.0016, SSIM: 0.9258
Model loaded from best_predrnn_model.pth
Validation Results: MSE: 0.0020, SSIM: 0.9201
Test Results: MSE: 0.0024, SSIM: 0.9065


In [ ]:

target_class = "Archery"
class_idx = label_encoder.transform([target_class])[0]
filtered_df = eval_df[eval_df["label_encoded"] == class_idx]

# Extract paths and labels for this class
filtered_paths = filtered_df["clip_path"].values
filtered_labels = filtered_df["label_encoded"].values


In [ ]:
filtered_dataset = VideoDataset(filtered_paths, filtered_labels, transform=transform)
filtered_loader = DataLoader(filtered_dataset, batch_size=4, shuffle=False)


In [ ]:
import numpy as np
import cv2
import torch

# Function to generate a video from frames
def generate_video(frames, save_path="generated_video.mp4", fps=10):
    # Check input frames
    if len(frames) == 0:
        print("Error: No frames to write!")
        return
    if frames.max() <= 1.0:
        frames = np.clip((frames * 255), 0, 255).astype(np.uint8)  # Scale to [0, 255]

    # Ensure frames are in [T, H, W, C] format
    print(f"Frames to write: {len(frames)}, Resolution: {frames[0].shape[:2]}")

    height, width = frames[0].shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(save_path, fourcc, fps, (width, height))

    # Write frames to video
    for i, frame in enumerate(frames):
        print(f"Writing frame {i+1}/{len(frames)}")
        video_writer.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))  # OpenCV expects BGR

    video_writer.release()
    print(f"Video saved to {save_path}")


# Load trained model
model.load_state_dict(torch.load("/content/best_predrnn_model.pth"))  # Path to your trained model
model.eval()

# Filtered loader for a specific class (this should be defined somewhere)
for batch_idx, (input_frames, target_frames) in enumerate(filtered_loader):
    input_frames = input_frames.to(device)  # Move input frames to the device

    # Predict frames using the model
    with torch.no_grad():
        pred_frames_batch = model(input_frames).cpu().numpy()  # Predicted frames of shape [batch_size, T, C, H, W]

    # Save videos for each batch
    for i, pred_frames in enumerate(pred_frames_batch):
        # Process the predicted frames
        pred_frames = pred_frames.transpose(0, 2, 3, 1)  # Convert to [T, H, W, C]
        pred_frames = (pred_frames * 0.5) + 0.5  # Convert to [0, 1]
        pred_frames = np.clip(pred_frames, 0, 1)  # Ensure frames are within [0, 1]

        # Generate the save path dynamically
        save_path = f"predicted_{target_class}_video_{batch_idx*4 + i + 1}.mp4"

        # Generate the video from predicted frames
        generate_video(pred_frames, save_path=save_path, fps=10)
        print(f"Saved: {save_path}")


<ipython-input-59-76b5e18b5685>:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/content/best_predrnn_model.pth"))  # Path to your traine

Frames to write: 5, Resolution: (64, 64)
Writing frame 1/5
Writing frame 2/5
Writing frame 3/5
Writing frame 4/5
Writing frame 5/5
Video saved to predicted_Archery_video_1.mp4
Saved: predicted_Archery_video_1.mp4
Frames to write: 5, Resolution: (64, 64)
Writing frame 1/5
Writing frame 2/5
Writing frame 3/5
Writing frame 4/5
Writing frame 5/5
Video saved to predicted_Archery_video_2.mp4
Saved: predicted_Archery_video_2.mp4
Frames to write: 5, Resolution: (64, 64)
Writing frame 1/5
Writing frame 2/5
Writing frame 3/5
Writing frame 4/5
Writing frame 5/5
Video saved to predicted_Archery_video_3.mp4
Saved: predicted_Archery_video_3.mp4
Frames to write: 5, Resolution: (64, 64)
Writing frame 1/5
Writing frame 2/5
Writing frame 3/5
Writing frame 4/5
Writing frame 5/5
Video saved to predicted_Archery_video_4.mp4
Saved: predicted_Archery_video_4.mp4
Frames to write: 5, Resolution: (64, 64)
Writing frame 1/5
Writing frame 2/5
Writing frame 3/5
Writing frame 4/5
Writing frame 5/5
Video saved to pr

In [ ]:
from google.colab import drive
from google.colab import files
import shutil
import os

# Mount Google Drive to access it from Colab
drive.mount('/content/drive')

# Upload files from local system
uploaded_files = files.upload()

# Define the folder name (new folder you want to create)
folder_name = 'dl_project_data'
destination_folder = f'/content/drive/MyDrive/{folder_name}/'

# Create the new folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Move uploaded files to the new folder
for filename in uploaded_files.keys():
    shutil.move(filename, destination_folder + filename)
    print(f"Moved {filename} to {destination_folder}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving Test.csv to Test (1).csv
Saving Train.csv to Train (1).csv
Saving Val.csv to Val (1).csv
Moved Test (1).csv to /content/drive/MyDrive/dl_project_data/
Moved Train (1).csv to /content/drive/MyDrive/dl_project_data/
Moved Val (1).csv to /content/drive/MyDrive/dl_project_data/


UI - PRED RNN

In [ ]:
from google.colab import files
import cv2
import numpy as np
import torch
from torchvision import transforms

# Define a file upload button for selecting video
def upload_video():
    print("Please upload your video file.")
    uploaded = files.upload()
    video_path = list(uploaded.keys())[0]  # Get the name of the uploaded file
    print(f"Video {video_path} uploaded successfully!")
    return video_path

# Preprocessing pipeline for input video frames
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((64, 64)),  # Resize to match model input dimensions
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize to mean 0.5 and std 0.5
])

# Function to extract frames from video
def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(transform(frame).unsqueeze(0))  # Add batch dimension
    cap.release()
    return torch.cat(frames, dim=0)  # Shape: [num_frames, C, H, W]

# Function to save predicted frames as a video
def generate_video(frames, save_path="predicted_video.mp4", fps=10):
    if len(frames) == 0:
        print("No frames to save!")
        return
    frames = (frames * 255).clip(0, 255).astype(np.uint8)  # Rescale to [0, 255]
    height, width = frames[0].shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(save_path, fourcc, fps, (width, height))
    for frame in frames:
        video_writer.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))  # Convert to BGR
    video_writer.release()
    print(f"Video saved to {save_path}")

# Initialize PredRNN model
input_frames = 10
pred_frames = 5
input_dim = 3
hidden_dim = 64
num_layers = 2
height, width = 64, 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PredRNN(input_dim, hidden_dim, num_layers, input_frames, pred_frames, height, width).to(device)

# Load the trained model state
model_path = "/content/best_predrnn_model.pth"
try:
    model.load_state_dict(torch.load(model_path, map_location=device))
    print(f"Model loaded successfully from {model_path}")
except Exception as e:
    print(f"Error loading model: {e}")

model.eval()

# Upload video
video_path = upload_video()

# Process video and predict future frames
input_video_frames = extract_frames(video_path)  # Shape: [num_frames, C, H, W]

# Ensure enough frames for input
if input_video_frames.size(0) < input_frames:
    raise ValueError("Input video does not contain enough frames for prediction.")

# Prepare input frames batch
input_batch = input_video_frames[:input_frames].unsqueeze(0).to(device)  # Shape: [1, T, C, H, W]

# Predict future frames
with torch.no_grad():
    predicted_frames = model(input_batch)  # Shape: [1, pred_frames, C, H, W]
    predicted_frames = predicted_frames.squeeze(0).permute(0, 2, 3, 1).cpu().numpy()  # Convert to [T, H, W, C]
    predicted_frames = (predicted_frames * 0.5) + 0.5  # Unnormalize to [0, 1]

# Save the predicted frames as a video
generate_video(predicted_frames, save_path="predicted_output.mp4", fps=10)
files.download("predicted_output.mp4")

Model loaded successfully from /content/best_predrnn_model.pth
Please upload your video file.


<ipython-input-63-3f4af0e60178>:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


Saving v_ApplyLipstick_g01_c05.avi to v_ApplyLipstick_g01_c05 (5).avi
Video v_ApplyLipstick_g01_c05 (5).avi uploaded successfully!
Video saved to predicted_output.mp4


UI - CONV_LSTM

In [ ]:
import os
import cv2
import torch
import numpy as np
from google.colab import files
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image
import torch.nn as nn
import torch.optim as optim

# Define a file upload button for selecting video
def upload_video():
    print("Please upload your video file.")
    uploaded = files.upload()
    video_path = list(uploaded.keys())[0]  # Get the name of the uploaded file
    print(f"Video {video_path} uploaded successfully!")
    return video_path

# Preprocessing pipeline for input video frames
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((64, 64)),  # Resize to match model input dimensions
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize to mean 0.5 and std 0.5
])

# Function to extract frames from video
def extract_frames(video_path, total_frames):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(transform(frame).unsqueeze(0))  # Add batch dimension
    cap.release()

    # Ensure the video has enough frames for input + prediction
    if len(frames) < total_frames:
        frames += [frames[-1]] * (total_frames - len(frames))  # Duplicate last frame if not enough frames
    return torch.cat(frames[:total_frames], dim=0)  # Shape: [num_frames, C, H, W]

# Function to save predicted frames as a video
def generate_video(frames, save_path="predicted_video.mp4", fps=10):
    if len(frames) == 0:
        print("No frames to save!")
        return
    frames = (frames * 255).clip(0, 255).astype(np.uint8)  # Rescale to [0, 255]
    height, width = frames[0].shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(save_path, fourcc, fps, (width, height))
    for frame in frames:
        video_writer.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))  # Convert to BGR
    video_writer.release()
    print(f"Video saved to {save_path}")

# Define the ConvLSTM model (simplified from the user's original code)
class ConvLSTM(nn.Module):
    def __init__(self, input_frames, pred_frames, num_classes=1):
        super(ConvLSTM, self).__init__()
        self.conv1 = nn.Conv3d(3, 32, kernel_size=(3, 3, 3), stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool3d((1, 2, 2))  # Reduces spatial dimensions (H, W) by half
        self.lstm = nn.LSTM(input_size=32 * 32 * 32, hidden_size=256, batch_first=True)
        self.fc = nn.Linear(256, pred_frames * 32 * 32 * 3)
        self.upsample = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=False)  # Restore to 64x64

    def forward(self, x):
        # Rearrange to match Conv3D input expectations
        x = x.permute(0, 2, 1, 3, 4)  # Shape: [batch_size, channels, depth, height, width]

        # Convolutional layers
        x = self.pool(self.relu(self.conv1(x)))  # Conv3D + Pooling

        # Flatten for LSTM
        b, c, t, h, w = x.size()
        x = x.view(b, t, -1)  # Shape: [batch_size, depth, features]

        # LSTM
        x, _ = self.lstm(x)  # Shape: [batch_size, depth, hidden_size]
        x = self.fc(x[:, -1, :])  # Use the last LSTM output

        # Reshape and upsample
        x = x.view(b, -1, 3, 32, 32)  # Shape: [batch_size, pred_frames, channels, 32, 32]
        x = x.view(-1, 3, 32, 32)  # Flatten pred_frames for upsampling
        x = self.upsample(x)  # Upsample to [batch_size * pred_frames, 3, 64, 64]
        x = x.view(b, -1, 3, 64, 64)  # Restore batch and pred_frames structure
        return x

# Initialize the model
input_frames = 10
pred_frames = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConvLSTM(input_frames, pred_frames).to(device)

# Load the trained model state
model_path = "/content/best_model.pth"  # Replace with your actual model path
try:
    model.load_state_dict(torch.load(model_path, map_location=device))
    print(f"Model loaded successfully from {model_path}")
except Exception as e:
    print(f"Error loading model: {e}")

model.eval()

# Upload video
video_path = upload_video()

# Process video and predict future frames
input_video_frames = extract_frames(video_path, total_frames=input_frames + pred_frames)  # Shape: [num_frames, C, H, W]

# Prepare input frames batch
input_batch = input_video_frames[:input_frames].unsqueeze(0).to(device)  # Shape: [1, T, C, H, W]

# Predict future frames
with torch.no_grad():
    predicted_frames = model(input_batch)  # Shape: [1, pred_frames, C, H, W]
    predicted_frames = predicted_frames.squeeze(0).permute(0, 2, 3, 1).cpu().numpy()  # Convert to [T, H, W, C]
    predicted_frames = (predicted_frames * 0.5) + 0.5  # Unnormalize to [0, 1]

# Save the predicted frames as a video
generate_video(predicted_frames, save_path="predicted_output.mp4", fps=10)

# Optionally, download the predicted video file
files.download("predicted_output.mp4")


Model loaded successfully from /content/best_model.pth
Please upload your video file.


<ipython-input-65-1ec6462a1d4b>:101: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


Saving v_ApplyLipstick_g01_c05.avi to v_ApplyLipstick_g01_c05 (6).avi
Video v_ApplyLipstick_g01_c05 (6).avi uploaded successfully!
Video saved to predicted_output.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>